### Import relevant packages

In [12]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

### Data

In [13]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))


## Model

### Outline the model

In [18]:
input_size = 784
output_size = 10
hidden_layer_size = 100

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

### Choose an Optimizer and loss function

In [19]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training

In [20]:
NUM_EPOCHS = 5

model.fit(train_data, epochs=NUM_EPOCHS,validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 5s - loss: 0.3397 - accuracy: 0.9032 - val_loss: 0.1757 - val_accuracy: 0.9468 - 5s/epoch - 10ms/step
Epoch 2/5
540/540 - 4s - loss: 0.1449 - accuracy: 0.9571 - val_loss: 0.1218 - val_accuracy: 0.9613 - 4s/epoch - 7ms/step
Epoch 3/5
540/540 - 4s - loss: 0.1046 - accuracy: 0.9679 - val_loss: 0.1099 - val_accuracy: 0.9655 - 4s/epoch - 7ms/step
Epoch 4/5
540/540 - 3s - loss: 0.0825 - accuracy: 0.9746 - val_loss: 0.0811 - val_accuracy: 0.9745 - 3s/epoch - 5ms/step
Epoch 5/5
540/540 - 4s - loss: 0.0669 - accuracy: 0.9789 - val_loss: 0.0721 - val_accuracy: 0.9782 - 4s/epoch - 7ms/step


### Test the model

In [41]:
test_loss, test_accuracy = model.evaluate(test_data)
print(f"Test loss: {test_loss:.2f}. Test accuracy: {(test_accuracy*100):.2f}%")

1/1 [==============================] - 0s 232ms/step - loss: 0.0918 - accuracy: 0.9709
Test loss: 0.09. Test accuracy: 97.09%
